In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Read Data Function
def read_data(filename):
    
    # Retrieve Data
    data = pd.read_csv(filename)
    
    # Drop Columns
    clean = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Date'])
    
    # Drop Last Row
    clean = clean[:-1].copy()
    
    # Drop missing data
    clean = clean.dropna()
    
    # Rename Columns
    cols = ['Year', 'Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']
    clean.columns = cols
    
    return clean

In [3]:
# Run Model (Machine Learning)
def run_model(clean):
    
    # Create Lag Column
    clean['nextN2O'] = clean.shift(1)['N2O']
    
    # Set Training and Testing Split
    training = clean.loc[(clean['Year']<2010) & (clean['Year']>1990)]
    testing = clean.loc[clean['Year']>=2010]

    # Set Variables
    X_train = training[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_train = training['nextN2O']
    X_test = testing[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_test = testing['nextN2O']
    X = clean[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y = clean['nextN2O']

    # Linear Regression Model
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()

    # Train Modele
    model.fit(X_train, y_train)

    # Score for 1991-2009
    print(f"Train Score: {model.score(X_train, y_train)}")

    # Score for 2010+
    print(f"Test Score: {model.score(X_test, y_test)}")

    # Predictions vs Actual
    predictions = model.predict(X)
    print(f"Actual: {y[1]}")
    print(f"Predicted: {predictions[1]}")
    print(f"Error: {predictions[1]-y[1]}")

    return pd.DataFrame({"Year": clean["Year"],
                         "Predicted": predictions,
                         "Actual": y,
                         "Error": predictions - y,
                         "Country": country})

In [4]:
final = pd.DataFrame(columns = ["Year", "Predicted", "Actual", "Error", "Country"])

for filename in os.listdir("Cleaning/Combined"):
    if ".csv" in filename:
        
        print()
        
        country = filename.replace('.csv', '')
        
        print(country)
        
        clean = read_data("Cleaning/Combined/" + filename)
        
        try:
            output = run_model(clean)
            final = final.append(output)
        
        except:
            print("Error")


Australia
Train Score: 0.9143527757985196
Test Score: -28.752075392069433
Actual: 15938.285071523998
Predicted: 15687.828907749827
Error: -250.45616377417173

Canada
Train Score: 0.8954811643644034
Test Score: -442.90936732417356
Actual: 38864.8794880575
Predicted: 38242.80513034691
Error: -622.0743577105895

Japan
Train Score: 0.8904209789888602
Test Score: -19.956362457731284
Actual: 26036.2447069511
Predicted: 25786.556123641523
Error: -249.68858330957664

New Zealand
Train Score: 0.9338694689995062
Test Score: -66.8335565142185
Actual: 18032.6596158077
Predicted: 18109.602190246333
Error: 76.94257443863171

Russia
Train Score: 0.7999692437698952
Test Score: -253.4475936817656
Actual: 15881.7433301115
Predicted: 14064.270428489312
Error: -1817.4729016221881

Spain
Train Score: 0.8705062404308335
Test Score: -6.530118291330219
Actual: 3326.81320982872
Predicted: 3351.9352718308396
Error: 25.122062002119492

Switzerland
Train Score: 0.9456685177466753
Test Score: -7.413325970207104
A

In [5]:
# Export 
final.to_csv('Resources/ML_N2O.csv')